## Library

In [1]:
import os
import numpy as np
import pandas as pd
import time
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from deap import base, creator, tools, algorithms

## Read File

In [2]:
def read_data(base_dir):
    """Reads all files in subdirectories of the given directory.
    
    Parameters:
        base_dir (str): Base directory to read all files inside.

    Returns:
        dict: Include Compound ID as key and Smiles as value.
    
    """
    #Save values in dictionary
    data_contents = {}

    # Check every subdicrectories
    for dir_name in os.listdir(base_dir):
        sub_dir_path = os.path.join(base_dir, dir_name)

        if os.path.isdir(sub_dir_path):
            file_path = os.path.join(sub_dir_path, "daylight-smiles")
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    data_contents[int(dir_name)] = f.read()  #Save file
    return data_contents

# Base directory
base_directory = "Data_GA/compounds"

# Read all file
data_files_content = read_data(base_directory)


In [3]:
# Read file
df = pd.read_table('Data_GA/properties/pIGC50/values')

# Change index 
df.set_index('Compound Id', inplace=True)

# Mapping df with Smiles data above base on Compound Id
df['Smiles'] = df.index.map(data_files_content)


In [4]:
df = df[['Smiles', 'pIGC50', ]]

df.to_csv('pIGC50.csv')

In [5]:
lflt = pd.read_csv('DATA_CHO_GA/compounds/compounds_with_LFLT.csv')

lflt = lflt[['SMILES', 'sdg']]

lflt.rename(columns={'SMILES': 'smiles'}, inplace= True)

lflt.to_csv('lflt.csv')

In [6]:
# Function to canonicalize SMILES strings
def canonical_smiles(smiles_list):
    """
    Convert a list of SMILES strings to their canonical forms.

    Parameters:
        smiles_list (list): List of SMILES strings.

    Returns:
        list: Canonicalized SMILES strings.
    """
    canonicalized = []
    for smi in smiles_list:
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol:
                canonicalized.append(Chem.MolToSmiles(mol))
            else:
                canonicalized.append(None)  # For invalid SMILES
        except Exception as e:
            canonicalized.append(None)  # Catch any exceptions
    return canonicalized

# Function to calculate RDKit molecular descriptors
def calculate_rdkit_descriptors(smiles_list):
    """
    Calculate molecular descriptors for a list of SMILES strings.

    Parameters:
        smiles_list (list): List of SMILES strings.

    Returns:
        tuple: A DataFrame of molecular descriptors and a list of descriptor names.
    """
    mols = [Chem.MolFromSmiles(smi) for smi in smiles_list if smi is not None]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    descriptors = []
    for mol in mols:
        try:
            # Add hydrogens to the molecule
            mol = Chem.AddHs(mol)
            # Calculate descriptors
            descriptors.append(calc.CalcDescriptors(mol))
        except Exception as e:
            descriptors.append([None] * len(desc_names))  # Handle errors by appending NaNs

    return pd.DataFrame(descriptors, columns=desc_names), desc_names


In [7]:
# Canonicalize SMILES strings
df['Canonical_SMILES'] = canonical_smiles(df['Smiles'])

# Calculate RDKit molecular descriptors
descriptors_df, descriptor_names = calculate_rdkit_descriptors(df['Canonical_SMILES'])

# Combine data with descriptors
result_df = pd.concat([df.reset_index(drop=True), descriptors_df], axis=1)

# Save the output to a CSV file

# output_file = 'pIGC50_descriptor.csv'
# result_df.to_csv(output_file, index=False)

# print(f"Descriptors saved to {output_file}")

In [8]:
lflt['Canonical_SMILES'] = canonical_smiles(lflt['smiles'])

# Calculate RDKit molecular descriptors
descriptors_df, descriptor_names = calculate_rdkit_descriptors(lflt['Canonical_SMILES'])

# Combine data with descriptors
lflt_descriptor = pd.concat([lflt.reset_index(drop=True), descriptors_df], axis=1)

lflt_descriptor

,smiles,sdg,Canonical_SMILES,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,O=C1CCCCC1C1CCCCC1,391.0,O=C1CCCCC1C1CCCCC1,13.150126,13.150126,2.655615,-4.691088,0.604646,97.692308,180.291,...,0,0,0,0,0,0,0,0,0,0
1,Nc1ccc(-c2ccccc2)cc1,440.0,Nc1ccc(-c2ccccc2)cc1,7.972286,7.972286,0.063020,-0.705730,0.652436,21.384615,169.227,...,0,0,0,0,0,0,0,0,0,0
2,CCC(C)C1CCCCC1,319.0,CCC(C)C1CCCCC1,8.425000,8.425000,4.041768,-4.636458,0.549343,89.600000,140.270,...,0,0,0,0,0,0,0,0,0,0
3,C=C/C=C/CC,240.0,C=C/C=C/CC,7.196412,7.196412,0.987847,-3.138015,0.447556,40.666667,82.146,...,0,0,0,0,0,0,0,0,0,0
4,C=C(C)/C=C/C,253.0,C=C(C)/C=C/C,7.251157,7.251157,1.033681,-2.950926,0.424989,40.666667,82.146,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,CCCCCCCCCCC=O,369.0,CCCCCCCCCCC=O,11.217636,11.217636,2.390479,-4.680877,0.380559,49.000000,170.296,...,0,0,0,0,0,0,0,0,0,0
1164,CCCCCCCCCCC,337.0,CCCCCCCCCCC,7.900978,7.900978,3.950866,-4.705017,0.452618,56.727273,156.313,...,0,0,0,0,0,0,0,0,0,0
1165,CCCCCCCCC(C)CC,346.0,CCCCCCCCC(C)CC,8.129337,8.129337,4.006342,-4.894467,0.459683,60.666667,170.340,...,0,0,0,0,0,0,0,0,0,0
1166,CCCCCCCCCCCN,368.0,CCCCCCCCCCCN,7.947853,7.947853,1.019382,-4.760980,0.528071,54.583333,171.328,...,0,0,0,0,0,0,0,0,0,0


## Preprocssing

In [9]:
result_df = result_df.dropna(axis=1)
lflt_descriptor = lflt_descriptor.dropna(axis=1)


In [10]:
X = result_df.iloc[:, 4:]
y = result_df.iloc[:, 1:2]


In [11]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [12]:
#lflt_descriptor.pop('pIGC50')

lflt_descriptor.to_csv('lflt_descriptor.csv')
result_df.to_csv('pIGC50_desriptor.csv')

In [13]:
lflt_descriptor['sdg'].describe()

count    1168.000000
mean      333.414108
std        67.318196
min       169.000000
25%       282.000000
50%       327.000000
75%       376.000000
max       559.000000
Name: sdg, dtype: float64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y.values, test_size=0.2, random_state=42)

In [15]:
type(X_train)

numpy.ndarray

## GA-MLR

In [16]:
# Genetic Algorithm parameters
n_population = 200  # Population size
n_generations = 100  # Number of generations

# Define the fitness function
def fitness(individual):
    selected_features = [i for i in range(len(individual)) if individual[i] == 1]
    if not selected_features:  # Penalize if no features are selected
        return 1e6,
    X_selected = X_train[:, selected_features]
    model = LinearRegression().fit(X_selected, y_train)
    y_pred = model.predict(X_selected)
    mse = mean_squared_error(y_train, y_pred)
    return mse,

# DEAP Setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 0, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_train.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

start_time = time.time()
# Run Genetic Algorithm
population = toolbox.population(n=n_population)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=n_generations, verbose=True)

# Best solution
best_individual = tools.selBest(population, k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1]

print("Selected features indices:", selected_features)

# Evaluate the final model on the test set
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

final_model = LinearRegression().fit(X_train_selected, y_train)

end_time = time.time()
run_time_ga_mlr = end_time - start_time

y_pred_ga_mlr= final_model.predict(X_test_selected)

mse_ga_mlr = mean_squared_error(y_test, y_pred_ga_mlr)
r2_ga_mlr = r2_score(y_test, y_pred_ga_mlr)

print("MSE GA-MLR model:", mse_ga_mlr)
print("R2 GA-MLR model:", r2_ga_mlr)
print("Run time model:", run_time_ga_mlr, "seconds")


gen	nevals
0  	200   
1  	120   
2  	114   
3  	112   
4  	125   
5  	128   
6  	124   
7  	110   
8  	119   
9  	117   
10 	108   
11 	131   
12 	127   
13 	116   
14 	118   
15 	129   
16 	117   
17 	124   
18 	113   
19 	121   
20 	123   
21 	127   
22 	126   
23 	119   
24 	119   
25 	110   
26 	118   
27 	116   
28 	125   
29 	125   
30 	113   
31 	114   
32 	130   
33 	137   
34 	134   
35 	124   
36 	126   
37 	111   
38 	118   
39 	119   
40 	124   
41 	119   
42 	122   
43 	124   
44 	117   
45 	114   
46 	126   
47 	125   
48 	119   
49 	122   
50 	113   
51 	134   
52 	124   
53 	118   
54 	105   
55 	113   
56 	103   
57 	118   
58 	143   
59 	115   
60 	117   
61 	116   
62 	113   
63 	117   
64 	120   
65 	115   
66 	117   
67 	133   
68 	113   
69 	111   
70 	111   
71 	122   
72 	113   
73 	121   
74 	116   
75 	124   
76 	119   
77 	123   
78 	113   
79 	128   
80 	122   
81 	111   
82 	125   
83 	121   
84 	118   
85 	120   
86 	99    
87 	108   
88 	127   
89 	126   

## PLS

In [17]:
#Call model with 50 components
pls = PLSRegression(n_components = 50) #can change n_components for better result

start_time = time.time()
#fit model
pls.fit(X_train, y_train)

end_time = time.time()

#predict
y_pred_pls= pls.predict(X_test)

#evaluate model
run_time_pls = end_time - start_time
mse_pls = mean_squared_error(y_test,y_pred_pls)
r2_pls = r2_score(y_test,y_pred_pls)

print(f"MSE PLS model: " ,mse_pls)
print(f"R2 PLS model: ", r2_pls)
print(f"Run time PLS model: ", run_time_pls)

MSE PLS model:  0.3172612980264073
R2 PLS model:  0.6992235750194586
Run time PLS model:  0.10117006301879883


## ANN

In [18]:
#Call model with (64,32) hidden layer
mlp = MLPRegressor(hidden_layer_sizes=(64,32))

start_time = time.time()

#fit model
mlp.fit(X_train, y_train)

end_time = time.time()

#predict
y_pred_mlp= mlp.predict(X_test)

#evalute model
run_time_mlp = end_time - start_time
mse_mlp = mean_squared_error(y_test,y_pred_mlp)
r2_mlp = r2_score(y_test,y_pred_mlp)

print(f"MSE ANN model: " ,mse_mlp)
print(f"R2 ANN model: ", r2_mlp)
print(f"Run time ANN model: ", run_time_mlp)

c:\Users\lalal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE ANN model:  0.2849132472557856
R2 ANN model:  0.7298908235190434
Run time ANN model:  1.3287100791931152


## Random Forest

In [19]:
#Call model
rf = RandomForestRegressor()

start_time = time.time()

#Fit model
rf.fit(X_train, y_train)

end_time = time.time()

#Predict
y_pred_rf= rf.predict(X_test)

#evalaute 
run_time_rf = end_time - start_time
mse_rf = mean_squared_error(y_test,y_pred_rf)
r2_rf = r2_score(y_test,y_pred_rf)

print(f"MSE Random Forest model: " ,mse_rf)
print(f"R2 Random Forest model: ", r2_rf)
print(f"Run time Random Forest model: ", run_time_rf)

c:\Users\lalal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


MSE Random Forest model:  0.2660280425423133
R2 Random Forest model:  0.7477947544241961
Run time Random Forest model:  7.4296486377716064


## Conclusion

In [20]:
# Tạo danh sách để lưu kết quả
models = []
mse_values = []
r2_values = []
run_times = []

In [21]:
models.extend(['GA-MLR', 'PLS', 'ANN', 'Random Forest'])
mse_values.extend([mse_ga_mlr, mse_pls, mse_mlp, mse_rf])
r2_values.extend([r2_ga_mlr,r2_pls,r2_mlp,r2_rf])
run_times.extend([run_time_ga_mlr,run_time_pls,run_time_mlp,run_time_pls])

In [ ]:
# Tạo DataFrame kết quả
compare_df = pd.DataFrame({
    'Model': models,
    'MSE': mse_values,
    'R2': r2_values,
    'Run Time (s)': run_times
})

compare_df


,Model,MSE,R2,Run Time (s)
0,GA-MLR,0.483960,0.541187,307.193437
1,PLS,0.317261,0.699224,0.101170
2,ANN,0.284913,0.729891,1.328710
3,Random Forest,0.266028,0.747795,0.101170
